# Setup

## Google Only

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import os
import sys
sys.path.append(os.path.dirname(os.path.abspath('')))
sys.path.append('/content/drive/My Drive/Code/autocomplete_me')

In [3]:
!ls -l '/content/drive/My Drive/Code/autocomplete_me/src'

total 19
-rw------- 1 root root 2516 Jul 12 14:10 predict_utils.py
drwx------ 2 root root 4096 Jul 11 13:20 __pycache__
-rw------- 1 root root 2943 Jul 12 15:50 reader.py
-rw------- 1 root root 9341 Jul 12 14:24 utils.py


In [4]:
from src import utils, reader, predict_utils

Using TensorFlow backend.


In [5]:
from importlib import reload
reload(utils)
reload(reader)
reload(predict_utils)

<module 'src.predict_utils' from '/content/drive/My Drive/Code/autocomplete_me/src/predict_utils.py'>

# Build Model

## Training

In [6]:
text = reader.read_alice()
content_type = 'Alice'

In [7]:
text[0]

'           ALICE’S ADVENTURES IN WONDERLAND  Lewis Carroll  THE MILLENNIUM FULCRUM EDITION 3'

In [9]:
len(text)

991

### Short Text

In [13]:
training_dict, word_idx, idx_word, sequences, num_words = utils.get_data(text, training_len=20, short=True)

There are 3659 unique words.
There are 13925 sequences.


In [14]:
len(training_dict)

4

In [15]:
# embedding_matrix = utils.create_embedding_matrix(word_idx, num_words, '/Users/jaipancholi/data/glove.6B.100d.txt')
embedding_matrix = utils.create_embedding_matrix(word_idx, num_words, '/content/drive/My Drive/Code/autocomplete_me/data/glove.6B.100d.txt')
embedding_matrix

Glove Vectors loading with dimension 100
There were 1361 words without pre-trained embeddings.


/content/drive/My Drive/Code/autocomplete_me/src/utils.py:180: RuntimeWarning: invalid value encountered in true_divide
  embedding_matrix = embedding_matrix / np.linalg.norm(embedding_matrix, axis=1).reshape((-1, 1))


array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.01938821,  0.01990321,  0.10770387, ..., -0.14973777,
         0.08155941,  0.0148697 ],
       [-0.00656124, -0.04206555,  0.12508174, ..., -0.02506376,
         0.14220549,  0.04648907],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

#### Unidirectional, 1 layer, Trainable

In [16]:
model = utils.make_word_level_model(
    num_words,
    embedding_matrix,
    lstm_cells=64,
    trainable=True,
    lstm_layers=1,
    bi_direc=False
)

In [17]:
history = utils.train_model(
    training_dict,
    f'{content_type}_uni-1_layer-trainable-20_seq',
    model=model,
    # use_pretrained_model=True,
    epochs=250
)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 9747 samples, validate on 4178 samples
Epoch 1/250
9747/9747 [==============================] - 3s 289us/step - loss: 8.1963 - accuracy: 0.0070 - val_loss: 8.1758 - val_accuracy: 0.0476
Epoch 2/250
9747/9747 [==============================] - 1s 66us/step - loss: 8.1396 - accuracy: 0.0330 - val_loss: 8.0671 - val_accuracy: 0.0481
Epoch 3/250
9747/9747 [==============================] - 1s 68us/step - loss: 7.9322 - accuracy: 0.0361 - val_loss: 7.6935 - val_accuracy: 0.0479
Epoch 4/250
9747/9747 [==============================] - 1s 66us/step - loss: 7.3538 - accuracy: 0.0363 - val_loss: 6.9723 - val_accuracy: 0.0479
Epoch 5/250
9747/9747 [==============================] - 1s 72us/step - loss: 6.5641 - accuracy: 0.0421 - val_loss: 6.5211 - val_accuracy: 0.0479
Epoch 6/250
9747/9747 [==============================] - 1s 66us/step - loss: 6.1974 - accuracy: 0.0488 - val_loss: 6.6320 - val_accuracy: 0.0479
Epoch 7/250
9747/9747 [==============================] - 1s 67us/step - los

## Compare Model

<tr>
    <th>Name of Model</th>
    <th>LTSM Layers</th>
    <th>Bidirectional</th>
    <th>Trainable Embeddings</th>
    <th>Sequence Length</th>
    <th>LSTM Dropout</th>
    <th>LSTM Recurrent Dropout</th>
    <th>Val Loss</th>
    <th>Val Accuracy</th>
</tr>
<!-- <tr>
    <td></td>
    <td>1</td>
    <td>False</td>
    <td>True</td>
    <td>50</td>
    <td>0.1</td>
    <td>0.1</td>
    <td>6.5235</td>
    <td>0.1715</td>
</tr>
<tr>
    <td></td>
    <td>1</td>
    <td>False</td>
    <td>True</td>
    <td>50</td>
    <td>0.2</td>
    <td>0.2</td>
    <td>6.4229</td>
    <td>0.1706</td>
</tr>
<tr>
    <td></td>
    <td>1</td>
    <td>False</td>
    <td>True</td>
    <td>50</td>
    <td>0.5</td>
    <td>0.5</td>
    <td>6.2247</td>
    <td>0.1638</td>
</tr>
<tr>
    <td></td>
    <td>1</td>
    <td>False</td>
    <td>True</td>
    <td>20</td>
    <td>0.1</td>
    <td>0.1</td>
    <td>6.4449</td>
    <td>0.1725</td>
</tr>
<tr>
    <td></td>
    <td>1</td>
    <td>False</td>
    <td>True</td>
    <td>20</td>
    <td>0.2</td>
    <td>0.2</td>
    <td>6.2899</td>
    <td>0.1706</td>
</tr>
<tr>
    <td></td>
    <td>1</td>
    <td>False</td>
    <td>True</td>
    <td>20</td>
    <td>0.5</td>
    <td>0.5</td>
    <td>6.1667</td>
    <td>0.1719</td>
</tr> -->

# Sample Output

## Load Model

In [18]:
from keras.models import load_model
# Get Model Weights and Architecture

# MODELS_DIR = os.path.join(os.path.dirname(os.path.abspath('')), 'models')
MODELS_DIR = '/content/drive/My Drive/Code/autocomplete_me/models'
model_filepath = os.path.join(MODELS_DIR, f'{content_type}_uni-1_layer-trainable-20_seq.h5')

model = load_model(model_filepath)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [19]:
# Get Text Data
TRAINING_LENGTH = 50
training_dict, word_idx, idx_word, sequences, num_words = utils.get_data(text, training_len=TRAINING_LENGTH)

There are 3659 unique words.
There are 4272 sequences.


In [20]:
original_sequence, gen_list, a = predict_utils.generate_output(
    model,
    sequences,
    idx_word,
    seed_length=TRAINING_LENGTH,
    new_words=50,
    diversity=1,
    n_gen=1
)

In [21]:
' '.join(word for word in original_sequence)

'was all finished , the Owl , as a boon , Was kindly permitted to pocket the spoon: While the Panther received knife and fork with a growl , And concluded the banquet-- ‘What IS the use of repeating all that stuff ,’ the Mock Turtle interrupted , ‘if you'

In [22]:
' '.join(word for word in gen_list[0])

'< --- > mouse who sobbing all added on them his for at they some knows crowd want second all hall faster WATCH that if , my into mark ever she thing ’ again Alice)--‘and at like fair Mock head have like same'

In [23]:
sentence = 'Stocks of major large technology firms are becoming'
predict_utils.generate_custom_sentence(sentence, word_idx, idx_word, model, new_words=50)

[None, 6, None, 130, None, None, 106, 3366]
‘It (look taking weeks ready not Mouse tell leaves told who advance dreadfully ‘All little it ‘It’s then door such or to put I’ll I were ,’ time when ‘’Tis door of an ‘Would up ?’ it’s the arches words better killing sitting do:-- purple perfectly do of she ‘You’ll
